In [ ]:
from pathlib import Path
import numpy as np
import pandas as pd
import scipy.ndimage as ndi
import cv2
import matplotlib.pyplot as plt
import os

def get_mask(rle, img_width, img_heigth):
    array = np.fromiter(rle.split(), dtype=np.uint)
    array = array.reshape((-1, 2)).T
    array[0] = array[0] - 1
    starts, lenghts = array
    mask_decompressed = np.concatenate([np.arange(s, s + l, dtype=np.uint) for s, l in zip(starts, lenghts)])
    mask_img = np.zeros(img_width * img_heigth, dtype=np.uint8)
    mask_img[mask_decompressed] = 1
    mask_img = mask_img.reshape((img_width, img_heigth))
    return mask_img

os.mkdir("./my_train_label") # 创建新的标签文件夹
train = pd.read_csv("../input/sartorius-cell-instance-segmentation/train.csv") # 读取train.csv文件
train_ids = train['id'].unique().tolist() # 获取样本图片即原始图的名字
for my_id in train_ids:
    annotations = train[train["id"] == my_id]['annotation'].tolist()
    mask_img = np.zeros((520,704), dtype=np.uint8)
    for annotation in annotations:
        now_mask = get_mask(annotation,520,704)
        mask_img = mask_img + now_mask
    cv2.imwrite("./my_train_label/" + my_id+".png",mask_img*255,[int(cv2.IMWRITE_PNG_COMPRESSION),0]) # 将得到的mask存储到my_train_label文件下名字与原图相同